In [2]:
import sympy as sym

In [3]:
sym.init_printing()
t,a,theta, v,p, z,n, u, k , l,sigma,tau,b,psi,d,e, x = sym.symbols('t alpha theta nu rho zeta eta mu kappa lambda sigma tau beta psi delta epsilon chi')

In [4]:
S = sym.Function('S')(t)
I = sym.Function('I')(t)
D = sym.Function('D')(t)
A = sym.Function('A')(t)
R = sym.Function('R')(t)
T = sym.Function('T')(t)
H = sym.Function('H')(t)
E = sym.Function('E')(t)
dS = S.diff(t)
expr1 = sym.Eq(dS,S*(a*I+b*D+psi*A+d*R)) 
dI = I.diff(t)
expr2 = sym.Eq(dI,S*(a*I+b*D+psi*A+d*R)-(e+z+l)*I)
dD = D.diff(t)
expr3 = sym.Eq(dD,e*I-(n+p)*D)
dA=A.diff(A)
expr4 = sym.Eq(dA,z*I-(theta+u+k)*A)
dR=R.diff(t)
expr5 = sym.Eq(dR,n*D+theta*A-(n+z)*R)
dT=T.diff(t)
expr6 = sym.Eq(dT,u*A+v*R-(sigma+tau)*T)
dH = H.diff(t)
expr7 = sym.Eq(dH, l*I+p*D+k*A+x*R+sigma*T)
dE = E.diff(t)
expr8 = sym.Eq(dE,tau*T)





In [15]:
system = [expr1,expr2]
sym.solvers.ode.systems.dsolve_system(system)

NotImplementedError: 
The system of ODEs passed cannot be solved by dsolve_system.

In [6]:
sym.dsolve(expr2)

KeyboardInterrupt: 